In [1]:
import torch
from peft import PeftModel    
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "meta-llama/CodeLlama-7b-Instruct-hf"
adapters_name = "codellama-instruct-repair"

m = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
m = PeftModel.from_pretrained(m, adapters_name)
m = m.merge_and_unload()
m.eval()

tok = AutoTokenizer.from_pretrained(model_name)
tok.pad_token = tok.eos_token

print(f"Successfully loaded the model {model_name} into memory")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Successfully loaded the model meta-llama/CodeLlama-7b-Instruct-hf into memory


In [2]:
def format_prompt(prompt):
    return f"<s>[INST] {prompt} [\INST]"

In [15]:
def generate_patch(prompt):
    with torch.no_grad():
        inputs = tok([format_prompt(prompt)], return_tensors="pt", max_length=4096, truncation=True)
        inputs = inputs.to("cuda")
        outputs = m.generate(**inputs, max_length=4096, stop_strings=["<\s>", "```<\s>"], tokenizer=tok)
        print(outputs)
        return tok.batch_decode(outputs, skip_special_tokens=False)[0].split("[\INST]")[1]

In [16]:
import json

bugs = []
with open("/home/x_andaf/elle-elle-aime/samples_defects4j_instruct_.jsonl", "r") as f:
    for line in f.readlines():
        bugs.append(json.loads(line))

bugs = [b for b in bugs if b["prompt"]]
print(len(bugs))

484


In [17]:
import tqdm

patches = []
for bug in tqdm.tqdm(bugs):
    patches.append(generate_patch(bug["prompt"]))
    break

print(patches[0])

  0%|                                                                                                                       | 0/484 [01:21<?, ?it/s]

tensor([[   1,    1,  518,  ..., 6425, 5015,  353]], device='cuda:0')
 ```java
    /**
     * Get an absolute URL from a URL attribute that may be relative (i.e. an <code>&lt;a href></code> or
     * <code>&lt;img src></code>).
     * <p/>
     * E.g.: <code>String absUrl = linkEl.absUrl("href");</code>
     * <p/>
     * If the attribute value is already absolute (i.e. it starts with a protocol, like
     * <code>http://</code> or <code>https://</code> etc), and it successfully parses as a URL, the attribute is
     * returned directly. Otherwise, it is treated as a URL relative to the element's {@link #baseUri}, and made
     * absolute using that.
     * <p/>
     * As an alternate, you can use the {@link #attr} method with the <code>abs:</code> prefix, e.g.:
     * <code>String absUrl = linkEl.attr("abs:href");</code>
     *
     * @param attributeKey The attribute key
     * @return An absolute URL if one could be made, or an empty string (not null) if the attribute was missing or